# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [3]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)
total = 0
for sqrt_value in generator:
    print(sqrt_value)
    # total += sqrt_value

# print(total)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [4]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [5]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [6]:
import dlt

## Append

In [7]:
# to do: homework :)

pipeline = dlt.pipeline(
                pipeline_name='dlt_hw_pl_1',
                destination='duckdb',
                dataset_name='dlt_hw')

# run the pipeline with default settings, and capture the outcome
info = pipeline.run(people_1(),
                    table_name="users",
                    write_disposition="append")

print(info)

info = pipeline.run(people_2(),
                    table_name="users",
                    write_disposition="append")

print(info)


Pipeline dlt_hw_pl_1 load step completed in 0.39 seconds
1 load package(s) were loaded to destination duckdb and into dataset dlt_hw
The duckdb destination used duckdb:////content/dlt_hw_pl_1.duckdb location to store data
Load package 1708933573.3882637 is LOADED and contains no failed jobs
Pipeline dlt_hw_pl_1 load step completed in 0.32 seconds
1 load package(s) were loaded to destination duckdb and into dataset dlt_hw
The duckdb destination used duckdb:////content/dlt_hw_pl_1.duckdb location to store data
Load package 1708933574.217506 is LOADED and contains no failed jobs


In [10]:

import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n users table below:")

rides = conn.sql("SELECT * FROM users").df()
display(rides)

sum_ages = conn.sql("SELECT SUM(age) FROM users").df()
print(sum_ages)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ users               │
└─────────────────────┘




 users table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708933573.3882637,8pxAxofCYbmP3Q,None
1,2,Person_2,27,City_A,1708933573.3882637,wIOSff79AHGBIg,None
2,3,Person_3,28,City_A,1708933573.3882637,hyFPpcoQosa4tQ,None
3,4,Person_4,29,City_A,1708933573.3882637,kPYCaItSIz/znw,None
4,5,Person_5,30,City_A,1708933573.3882637,Boivr3L8YMRO+w,None
5,3,Person_3,33,City_B,1708933574.217506,roCxdf7r/erCXA,Job_3
6,4,Person_4,34,City_B,1708933574.217506,vcHuVBUGYR9e+g,Job_4
7,5,Person_5,35,City_B,1708933574.217506,9sM+xTJebAHchg,Job_5
8,6,Person_6,36,City_B,1708933574.217506,kVzFoWFemT6Gow,Job_6
9,7,Person_7,37,City_B,1708933574.217506,KE9Fz9SVWtogKA,Job_7


   sum(age)
0     353.0


## Merge

In [16]:
pipeline2 = dlt.pipeline(
                pipeline_name='dlt_hw_pl_2',
                destination='duckdb',
                dataset_name='dlt_hw2')

# run the pipeline with default settings, and capture the outcome
info = pipeline2.run(people_1(),
                    table_name="users3",
                    write_disposition="merge",
                    primary_key="record_hash")

print(info)

info = pipeline2.run(people_2(),
                    table_name="users3",
                    write_disposition="merge",
                    primary_key="record_hash")

print(info)

2024-02-26 07:59:23,366|[WARNING              ]|398|140461009002496|dlt|reference.py|_verify_schema:357|A column record_hash in table users3 in schema dlt_hw_pl_2 is incomplete. It was not bound to the data during normalizations stage and its data type is unknown. Did you add this column manually in code ie. as a merge key?
2024-02-26 07:59:23,448|[WARNING              ]|398|140461009002496|dlt|reference.py|_verify_schema:357|A column record_hash in table users3 in schema dlt_hw_pl_2 is incomplete. It was not bound to the data during normalizations stage and its data type is unknown. Did you add this column manually in code ie. as a merge key?


Pipeline dlt_hw_pl_2 load step completed in 0.44 seconds
1 load package(s) were loaded to destination duckdb and into dataset dlt_hw2
The duckdb destination used duckdb:////content/dlt_hw_pl_2.duckdb location to store data
Load package 1708934362.9798808 is LOADED and contains no failed jobs


2024-02-26 07:59:24,258|[WARNING              ]|398|140461009002496|dlt|reference.py|_verify_schema:357|A column record_hash in table users3 in schema dlt_hw_pl_2 is incomplete. It was not bound to the data during normalizations stage and its data type is unknown. Did you add this column manually in code ie. as a merge key?
2024-02-26 07:59:24,369|[WARNING              ]|398|140461009002496|dlt|reference.py|_verify_schema:357|A column record_hash in table users3 in schema dlt_hw_pl_2 is incomplete. It was not bound to the data during normalizations stage and its data type is unknown. Did you add this column manually in code ie. as a merge key?


Pipeline dlt_hw_pl_2 load step completed in 0.57 seconds
1 load package(s) were loaded to destination duckdb and into dataset dlt_hw2
The duckdb destination used duckdb:////content/dlt_hw_pl_2.duckdb location to store data
Load package 1708934363.8445811 is LOADED and contains no failed jobs


In [17]:
conn = duckdb.connect(f"{pipeline2.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline2.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n users3 table below:")

rides = conn.sql("SELECT * FROM users3").df()
display(rides)

sum_ages = conn.sql("SELECT SUM(age) FROM users3").df()
print(sum_ages)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ users2              │
│ users3              │
└─────────────────────┘




 users3 table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,3,Person_3,33,City_B,1708934363.8445811,ngv/YOsqdVs9mA,Job_3
1,4,Person_4,34,City_B,1708934363.8445811,PB5vmUhZfEstsg,Job_4
2,5,Person_5,35,City_B,1708934363.8445811,R2SGliF06XQRPQ,Job_5
3,6,Person_6,36,City_B,1708934363.8445811,KEDMrMmLJE/udg,Job_6
4,7,Person_7,37,City_B,1708934363.8445811,IWOp9mubboaACg,Job_7
5,8,Person_8,38,City_B,1708934363.8445811,Kst34ayZin/2SQ,Job_8


   sum(age)
0     213.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX